# TensorFlow 기본 구조 이해

In [1]:
import tensorflow as tf

print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


## TensorFlow Programming Stack

<img src="../figures/tensorflow_programming_environment.png" width="70%">

## Execution Style

* [Eager execution](https://www.tensorflow.org/get_started/eager)
  * 최근에 나온 방식
  * 변수값을 바로 확인할 수 있음
  * 마치 `numpy` 짜듯이 짤 수 있음
* [Graph execution](https://www.tensorflow.org/get_started/get_started_for_beginners)
  * TensorFlow 초창기 구현 방법
  * 여전히 많이 이용하고 있음
  * Graph 선언부분과 실행(Session)하는 부분이 분리되어 있음
    * Phase1: Graph 선언
    * Phase2: Session

### Graph 생성

* 변수를 생성하고 계산을 한다는 것은 그래프를 만드는 과정
* `a`를 출력하면 **계산된 값**이 나오지 않음

<img src="../figures/TF_graph.png" width="60%">

In [2]:
a = tf.add(3, 5)

In [3]:
print(a)

Tensor("Add:0", shape=(), dtype=int32)


<img src="../figures/TF_add1.png" width="40%">

### Session 실행

* 위의 과정은 그래프 형태만 만들어놓음
* 실제 계산은 `tf.Session()`을 실행하여 계산함
* 마치 파이프에 물(데이터)을 흘려보내는 것과 비슷함
* `tf.Session()`을 열면 TF default로 GPU 메모리를 다 잡아버림
  * 그것을 방지하기 위해 `gpu_options`을 다음과 같이 준다
* GPU 사용량 확인하는 명령어
  * `nvidia-smi`
  * `watch`라는 명령어와 함께 쓰면 계속 갱신하면서 메모리 변화를 볼 수 있음
    * `watch -n 1 nvidia-smi`

In [4]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#sess = tf.Session()
sess = tf.Session(config=sess_config)
print(sess.run(a))
sess.close()

8


<img src="../figures/TF_add2.png" width="40%">

### `tf.Session()`을 `with` 구문으로

* `session`을 열면 `sess.close()`로 명시적으로 닫아줘야 한다.
* `with` 구문이 끝나면 알아서 `session`이 닫힌다.

In [5]:
with tf.Session(config=sess_config) as sess:
  print(sess.run(a))

8


### `with tf.Session()`

* 구문 안에서는 `sess.run()`대신 `eval()` method를 쓸 수 있다

In [6]:
with tf.Session(config=sess_config) as sess:
  print(a.eval())

8


### `tf.InteractiveSession()`

* `with`구문이 아니더라도 `eval()` method를 쓸 수 있다

In [7]:
sess = tf.InteractiveSession(config=sess_config)
print(a.eval())

8


### 약간 더 복잡한 계산

In [8]:
x = 2
y = 3
w = tf.add(x, y)
z = tf.multiply(x, y)
p = tf.pow(z, w)
print(p)
with tf.Session(config=sess_config) as sess:
  print(sess.run(p))

Tensor("Pow:0", shape=(), dtype=int32)
7776


<img src="../figures/TF_more_calculation.png" width="60%">

### `Tensor`변수와 일반 `python` 변수

In [9]:
import numpy as np

x = 2
y = 3
w = x + y
z = x * y
p = np.power(z, w)
print(p)
with tf.Session(config=sess_config) as sess:
  print(sess.run(p)) # p는 sess.run()에 넣는 Tensor가 아니고 일반 python 변수이기 때문에 에러가 난다

7776


TypeError: Fetch argument 7776 has invalid type <class 'numpy.int64'>, must be a string or Tensor. (Can not convert a int64 into a Tensor or Operation.)

In [10]:
x = tf.constant(2)
y = tf.constant(3)
w = x + y
z = x * y
p = tf.pow(z, w)
print(p)
with tf.Session(config=sess_config) as sess:
  print(sess.run(p))

Tensor("Pow_1:0", shape=(), dtype=int32)
7776
